In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''



now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

In [2]:
maxpage = 10
query = "금리"
sort = '1' #관련도순=0  최신순=1  오래된순=2
s_date = '2018.01.01'  #2019.01.04
e_date = '2019.12.31'  #2019.01.05

#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

In [3]:

s_from = s_date.replace(".","")
e_to = e_date.replace(".","")
page = 1  
maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

while page <= maxpage_t:
    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)

    print(url)
    response = requests.get(url)
    html = response.text

    #뷰티풀소프의 인자값 지정
    soup = BeautifulSoup(html, 'html.parser')

    #<a>태그에서 제목과 링크주소 추출
    atags = soup.select('._sp_each_title')
    for atag in atags:
        title_text.append(atag.text)     #제목
        link_text.append(atag['href'])   #링크주소

    #신문사 추출
    source_lists = soup.select('._sp_each_source')
    for source_list in source_lists:
        source_text.append(source_list.text)    #신문사

    #날짜 추출 
    date_lists = soup.select('.txt_inline')
    for date_list in date_lists:
        test=date_list.text   
        date_cleansing(test)  #날짜 정제 함수사용 

    #본문요약본
    contents_lists = soup.select('ul.type01 dl')
    for contents_list in contents_lists:
        #print('==='*40)
        #print(contents_list)
        contents_cleansing(contents_list) #본문요약 정제화


    #모든 리스트 딕셔너리형태로 저장
    result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
    #print(page)

    df = pd.DataFrame(result)  #df로 변환
    page += 10

https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=1
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=11
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=21
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=31
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=41
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp%3Afrom20180101to20191231%2Ca%3A&start=51
https://search.naver.com/search.naver?where=news&query=금리&sort=1&ds=2018.01.01&de=2019.12.31&nso=so%3Ar%2Cp

In [4]:
df

,date,title,source,contents,link
0,4분,코스피 상승 출발…외인·기관 동반 매수,연합인포맥스,전일 미국 국채 10년물과 3개월물의 금리가 큰 폭으로 역전되면서 경기 침체에 대한...,http://news.einfomax.co.kr/news/articleView.ht...
1,6분,"[오늘의 키워드] 美 역공 나선 中, 반격 카드인 ‘희토류’가 뭐길래?",SBS CNBC,- 美10년물 금리 하락…장단기 금리역전 또 발생 - 글로벌 경제 불확실성 고조…안...,https://cnbc.sbs.co.kr/article_hub/10000943238...
2,6분,"2019년 5월 30일 환율전망(한국무역보험공사, www.ksure.or.kr)",한국무역신문,"간밤 미국 10년 만기 국채수익률이 연방준비제도의 기준금리 하단을 하회했고, 3개월...",http://jedam15.koreafree.co.kr/news/view.html?...
3,6분,"부산은행-신보, 사회적경제기업 금융지원 업무협약 체결",매일일보,부산은행은 사회적경제기업들의 금융비용 부담 완화를 위해 대출금리를 최대 0.5%까지...,http://www.m-i.kr/news/articleView.html?idxno=...
4,10분,"경북도, 관광진흥기금 1,000억원 조성해 업계 지원",뉴스웨이,"융자사업은 관광관련시설의 신축, 증축, 개보수 비용으로 담보능력에 따라 최대 5억원...",http://www.newsway.co.kr/news/view?tp=1&ud=201...
5,10분,"[해외시황] 뉴욕, 무역전쟁 장기화·경기침체 우려에 하락…유럽↓",프라임경제,대표적인 경기 침체 신호인 미국 국채 10년물과 3개월물의 금리가 큰 폭으로 역전됐...,http://www.newsprime.co.kr/news/article.html?n...
6,14분,"코스피, 그간 많이 빠졌나…외국인 닷새 만에 '사자'",아시아경제,경기 침체 신호로 여기는 미국 국채 10년물과 3개월물의 금리가 큰 폭으로 역전됐다...,https://view.asiae.co.kr/news/view.htm?idxno=2...
7,16분,[이지 돋보기] 카드사의 불편한 고객 관리…결제 고객 '혜택 펑펑'·대출자 '...,이지경제,반면 금융서비스 이용 고객에게는 15~20%의 고금리를 물리면서 혜택은 전무한 것....,http://www.ezyeconomy.com/news/articleView.htm...
8,18분,금통위 금리 결정과 달러-원·FX스와프 시나리오,연합인포맥스,"이들은 대부분 기준금리가 동결될 것으로 보면서도, 금리 인하 소수의견이 나올 가능성...",http://news.einfomax.co.kr/news/articleView.ht...
9,20분,"삼성선물 ""미중 무역긴장, R의 공포…환율 1190원대 등락""",아주경제,"전승지 삼성선물 연구원은 ""미중 무역긴장 고조가 글로벌 경기침체를 재촉할 것이라는 ...",http://www.ajunews.com/view/20190530085656726
